### Maestría: Computación de Alto Desempeño

##### Autor: **Sebastián Acosta**
##### Tema: **Técnicas de Machine Learning Supervisado con el ecosistema Apache Spark en Google Colab**




# Procesamiento de Datos a Gran Escala con PySpark


1. Preparación del entorno de ejecución en Google Colab (Java, Spark, PySpark y configuración inicial).
2. Carga y exploración del dataset **Titanic**, ampliamente usado en problemas de clasificación.
3. Limpieza y preparación de los datos (selección de variables, tratamiento de valores faltantes y codificación categórica).
4. Construcción del *pipeline* de ML en PySpark, ensamblando las características en un vector numérico.
5. Entrenamiento de un modelo de Regresión Logística para predecir la supervivencia de los pasajeros.
6. Evaluación del modelo y análisis de resultados.



## 1. Configuración del entorno en Google Colab

Google Colab no trae Apache Spark preinstalado, por lo que es necesario instalar **Java**, descargar una distribución de **Spark** y luego instalar la librería `pyspark` junto con `findspark`.


In [ ]:
# Instalar Java (requerido para Spark)
!apt-get update > /dev/null 2>&1
!apt-get install -y openjdk-8-jdk > /dev/null 2>&1
!update-alternatives --install /usr/bin/java java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java 1 > /dev/null 2>&1

In [ ]:
# Instalar PySpark
!pip install pyspark==3.5.0 > /dev/null 2>&1
!pip install findspark > /dev/null 2>&1

In [ ]:
import findspark
import os
import warnings

# Suprimir warnings
warnings.filterwarnings('ignore')

# Configurar variables de entorno para Spark
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

findspark.init()

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

In [ ]:
# Crear sesión de Spark optimizada para Colab
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("ML_Supervisado_Titanic_SebastianAcosta") \
    .config("spark.driver.memory", "2g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.scheduler.mode", "FAIR") \
    .getOrCreate()

# Suprimir logs de Spark
spark.sparkContext.setLogLevel("ERROR")
spark

## 2. Carga y exploración del dataset Titanic

  
En esta sección se cargan los datos a partir de la librería `seaborn`, se convierten a un DataFrame de Spark y se realiza una exploración inicial para entender su estructura.


In [ ]:
# En esta celda se carga el dataset Titanic usando la función `load_dataset` de `seaborn`, se convierte a un DataFrame de Spark.
import seaborn as sns
import pandas as pd

titanic_pd = sns.load_dataset("titanic")
# Se eliminan filas sin información de la variable objetivo 'survived'
titanic_pd = titanic_pd.dropna(subset=["survived"])

df_titanic = spark.createDataFrame(titanic_pd)

df_titanic.printSchema()
df_titanic.show(5)

root
 |-- survived: long (nullable = true)
 |-- pclass: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: double (nullable = true)
 |-- sibsp: long (nullable = true)
 |-- parch: long (nullable = true)
 |-- fare: double (nullable = true)
 |-- embarked: string (nullable = true)
 |-- class: string (nullable = true)
 |-- who: string (nullable = true)
 |-- adult_male: boolean (nullable = true)
 |-- deck: string (nullable = true)
 |-- embark_town: string (nullable = true)
 |-- alive: string (nullable = true)
 |-- alone: boolean (nullable = true)

+--------+------+------+----+-----+-----+-------+--------+-----+-----+----------+----+-----------+-----+-----+
|survived|pclass|   sex| age|sibsp|parch|   fare|embarked|class|  who|adult_male|deck|embark_town|alive|alone|
+--------+------+------+----+-----+-----+-------+--------+-----+-----+----------+----+-----------+-----+-----+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|Third|  man|      true| NaN|Southampton|   n

In [ ]:
# En esta celda se calcula estadísticas descriptivas básicas sobre algunas columnas numéricas para entender rangos y distribuciones de los datos.
numeric_cols = ["age", "fare", "pclass", "sibsp", "parch"]
df_titanic.select(numeric_cols).describe().show()

+-------+----+------------------+------------------+------------------+-------------------+
|summary| age|              fare|            pclass|             sibsp|              parch|
+-------+----+------------------+------------------+------------------+-------------------+
|  count| 891|               891|               891|               891|                891|
|   mean| NaN|32.204207968574615| 2.308641975308642|0.5230078563411896|0.38159371492704824|
| stddev| NaN| 49.69342859718091|0.8360712409770493| 1.102743432293432| 0.8060572211299484|
|    min|0.42|               0.0|                 1|                 0|                  0|
|    max| NaN|          512.3292|                 3|                 8|                  6|
+-------+----+------------------+------------------+------------------+-------------------+



## 3. Preparación de los datos para el modelo supervisado

En esta fase se selecciona las columnas relevantes para el modelo de clasificación y maneja los valores faltantes.  
Además, realiza la codificación de variables categóricas (por ejemplo, `sex` y `embarked`) mediante `StringIndexer` y `OneHotEncoder`, y finalmente se construye el vector de características que se usará en el modelo.


In [ ]:
# En esta celda se selecciona las columnas de interés, se elimina filas con valores faltantes en esas columnas y se crea un nuevo DataFrame depurado.
from pyspark.sql.functions import col

feature_cols = ["pclass", "sex", "age", "fare", "embarked"]
target_col = "survived"

df_model = df_titanic.select([target_col] + feature_cols)

# Se eliminan filas con valores nulos o NaN en las variables seleccionadas
# na.drop() es más robusto para manejar tanto null como NaN en columnas numéricas.
df_model = df_model.na.drop(subset=feature_cols + [target_col])

df_model.show(5)
df_model.printSchema()

+--------+------+------+----+-------+--------+
|survived|pclass|   sex| age|   fare|embarked|
+--------+------+------+----+-------+--------+
|       0|     3|  male|22.0|   7.25|       S|
|       1|     1|female|38.0|71.2833|       C|
|       1|     3|female|26.0|  7.925|       S|
|       1|     1|female|35.0|   53.1|       S|
|       0|     3|  male|35.0|   8.05|       S|
+--------+------+------+----+-------+--------+
only showing top 5 rows

root
 |-- survived: long (nullable = true)
 |-- pclass: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: double (nullable = true)
 |-- fare: double (nullable = true)
 |-- embarked: string (nullable = true)



In [ ]:
# En esta celda el autor ensambla todas las características numéricas y categóricas en una sola columna tipo vector llamada `features`, usando `VectorAssembler`.
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["pclass", "age", "fare", "sex_ohe", "embarked_ohe"],
    outputCol="features"
)

df_final = assembler.transform(df_model).select(target_col, "features")
df_final.show(5)

In [ ]:
# En esta celda se aplica `StringIndexer` para convertir columnas categóricas (`sex` y `embarked`) a índices numéricos, lo cual es requerido por los algoritmos de PySpark.
from pyspark.ml.feature import StringIndexer

indexers = [
    StringIndexer(inputCol="sex", outputCol="sex_indexed"),
    StringIndexer(inputCol="embarked", outputCol="embarked_indexed")
]

for indexer in indexers:
    df_model = indexer.fit(df_model).transform(df_model)

df_model.select("sex", "sex_indexed", "embarked", "embarked_indexed").show(5)

+------+-----------+--------+----------------+
|   sex|sex_indexed|embarked|embarked_indexed|
+------+-----------+--------+----------------+
|  male|        0.0|       S|             0.0|
|female|        1.0|       C|             1.0|
|female|        1.0|       S|             0.0|
|female|        1.0|       S|             0.0|
|  male|        0.0|       S|             0.0|
+------+-----------+--------+----------------+
only showing top 5 rows



In [ ]:
# En esta celda se utiliza `OneHotEncoder` para transformar los índices categóricos en vectores dispersos, lo que permite representar correctamente las variables categóricas en el modelo.
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(
    inputCols=["sex_indexed", "embarked_indexed"],
    outputCols=["sex_ohe", "embarked_ohe"]
)

df_model = encoder.fit(df_model).transform(df_model)
df_model.select("sex_indexed", "sex_ohe", "embarked_indexed", "embarked_ohe").show(5)

+-----------+-------------+----------------+-------------+
|sex_indexed|      sex_ohe|embarked_indexed| embarked_ohe|
+-----------+-------------+----------------+-------------+
|        0.0|(1,[0],[1.0])|             0.0|(3,[0],[1.0])|
|        1.0|    (1,[],[])|             1.0|(3,[1],[1.0])|
|        1.0|    (1,[],[])|             0.0|(3,[0],[1.0])|
|        1.0|    (1,[],[])|             0.0|(3,[0],[1.0])|
|        0.0|(1,[0],[1.0])|             0.0|(3,[0],[1.0])|
+-----------+-------------+----------------+-------------+
only showing top 5 rows



In [ ]:
# En esta celda se ensambla todas las características numéricas y categóricas en una sola columna tipo vector llamada `features`, usando `VectorAssembler`.
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["pclass", "age", "fare", "sex_ohe", "embarked_ohe"],
    outputCol="features"
)

df_final = assembler.transform(df_model).select(target_col, "features")
df_final.show(5)

+--------+--------------------+
|survived|            features|
+--------+--------------------+
|       0|[3.0,22.0,7.25,1....|
|       1|[1.0,38.0,71.2833...|
|       1|[3.0,26.0,7.925,0...|
|       1|[1.0,35.0,53.1,0....|
|       0|[3.0,35.0,8.05,1....|
+--------+--------------------+
only showing top 5 rows



## 4. Entrenamiento del modelo supervisado

Una vez construidas las características, el autor divide los datos en conjuntos de entrenamiento y prueba.  
El modelo elegido es **Regresión Logística**, una técnica estándar para problemas de clasificación binaria.  
Posteriormente se entrena el modelo y se generan predicciones sobre el conjunto de prueba.


In [ ]:
# En esta celda se separa los datos en conjuntos de entrenamiento y prueba utilizando una partición aleatoria estratificada.
train_df, test_df = df_final.randomSplit([0.8, 0.2], seed=42)
print("Tamaño entrenamiento:", train_df.count())
print("Tamaño prueba:", test_df.count())

Tamaño entrenamiento: 588
Tamaño prueba: 126


In [ ]:
# En esta celda se entrena un modelo de Regresión Logística sobre el conjunto de entrenamiento y muestra un resumen básico del proceso de entrenamiento.
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(
    labelCol=target_col,
    featuresCol="features",
    maxIter=20,
    regParam=0.01
)

lr_model = lr.fit(train_df)

print("Coeficientes del modelo:")
print(lr_model.coefficients)
print("Intercepto:", lr_model.intercept)

Coeficientes del modelo:
[-1.078279170524033,-0.028276667221918163,0.001262573205555241,-2.1733616985633977,-0.2906547899158478,0.18606777057028018,-0.5373271216833035]
Intercepto: 4.2738673452053515


## 5. Evaluación del modelo y análisis de resultados

Se evalúa el desempeño del modelo de Regresión Logística utilizando una métrica de clasificación binaria (área bajo la curva ROC).  
Además, se revisa algunas predicciones individuales para interpretar mejor el comportamiento del modelo.


In [ ]:
# En esta celda se genera predicciones sobre el conjunto de prueba y muestra algunas filas para observar la columna de probabilidad y la etiqueta predicha.
predictions = lr_model.transform(test_df)
predictions.select("survived", "probability", "prediction").show(10, truncate=False)

+--------+----------------------------------------+----------+
|survived|probability                             |prediction|
+--------+----------------------------------------+----------+
|0       |[0.4414343613595366,0.5585656386404634] |1.0       |
|0       |[0.32918831755975864,0.6708116824402414]|1.0       |
|0       |[0.3727834389925769,0.6272165610074232] |1.0       |
|0       |[0.5985663230339688,0.40143367696603116]|0.0       |
|0       |[0.6269217001800422,0.37307829981995777]|0.0       |
|0       |[0.6483248113338012,0.35167518866619885]|0.0       |
|0       |[0.7213180129926975,0.2786819870073025] |0.0       |
|0       |[0.6983122989676245,0.3016877010323755] |0.0       |
|0       |[0.23786368875087677,0.7621363112491233]|1.0       |
|0       |[0.7383114528677232,0.26168854713227685]|0.0       |
+--------+----------------------------------------+----------+
only showing top 10 rows



In [ ]:
# En esta celda se calcula el área bajo la curva ROC (AUC) usando `BinaryClassificationEvaluator` para cuantificar el desempeño del modelo.
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(
    labelCol=target_col,
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)

auc = evaluator.evaluate(predictions)
print(f"AUC del modelo de Regresión Logística: {auc:.4f}")

AUC del modelo de Regresión Logística: 0.8776


## 6. Conclusiones del cuaderno supervisado

En este cuaderno:

-Se realizó la configuración de un entorno de trabajo en Google Colab para ejecutar PySpark, estableciendo las bases necesarias para el procesamiento y análisis de los datos.

-Se prepararon los datos mediante la selección de variables, el manejo de valores faltantes y la codificación de variables categóricas, garantizando así un conjunto de datos adecuado para el modelado.

-Se entrenó un modelo de Regresión Logística con el objetivo de predecir la probabilidad de supervivencia de los pasajeros.

-Se evaluó el desempeño del modelo utilizando el AUC, lo que permitió interpretar su capacidad para distinguir entre pasajeros que sobreviven y los que no.


